In [1]:
import csv
import ast
from itertools import combinations_with_replacement
import snappy

### braid operations

def mirror(braid_word):
    return([-x for x in braid_word])
def reverse_orientation(braid_word):
    return(braid_word[::-1])
def connected_sum(w1,w2):
    n=max([abs(k) for k in w1])
    w3=[k+n if k>0 else -(-k+n) for k in w2]
    return(w1+w3)

### ---- Load Invariants ----

def load_csv_as_dict(path, key_index=0, val_index=1, eval_value=True, cast=int):
    data = []
    with open(path, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            try:
                key = row[key_index]
                val = row[val_index]
                if eval_value:
                    val = ast.literal_eval(val)
                if cast:
                    val = cast(val)
                data.append((key, val))
            except:
                continue
    return dict(data)

# Paths
base_path = '/Users/leomousseau/Desktop/slice genera part 2/census seminar github'
group_path = base_path + '/group_A/computed results'
knot_floer_path = '/Users/leomousseau/Desktop/slice genera part 2/knot_floer.csv'
braid_path = base_path + '/braid_words.txt'

# Load braid words
with open(braid_path, 'r') as f:
    braid_words_list = ast.literal_eval(f.read())
braid_dict = dict(braid_words_list)
names = list(braid_dict.keys())

# Load invariants
s_dict = load_csv_as_dict('/Users/leomousseau/Downloads/sinvariants.csv')
sign_dict = load_csv_as_dict(group_path + '/signature.csv', eval_value=True, cast=int)

# From knot floer
knot_floer = []
with open(knot_floer_path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        knot_floer.append(row)

tau_dict = {}
eps_dict = {}
for row in knot_floer:
    name = row[0]
    try:
        tau_dict[name] = int(ast.literal_eval(row[7]))
    except:
        pass
    try:
        eps_dict[name] = int(ast.literal_eval(row[2]))
    except:
        pass

### ---- Concordance Filtering ----

variant_types = [
    ('identity', False),
    ('reverse', False),
    ('mirror', True),
    ('reverse_mirror', True)
]

slice_census = [
    'm032', 'm222', 'm372', 's704', 's879', 'v1971', 'v2272', 'v2362', 'v2508', 'v2543', 'v2861', 'v3195',
    'v3423', 'v3536', 't07281', 't07452', 't07599', 't08306', 't08617', 't10887', 't10974', 't11248', 
    't11418', 't11532', 't11577', 't12200', 't12587', 't12607', 't12630', 't12748', 'o9_12414', 'o9_18772', 
    'o9_18931', 'o9_20458', 'o9_20894', 'o9_22066', 'o9_29992', 'o9_30430', 'o9_31828', 'o9_34949', 
    'o9_35240', 'o9_36180', 'o9_36357', 'o9_36459', 'o9_37618', 'o9_37770', 'o9_38600', 'o9_39433', 
    'o9_39806', 'o9_39930', 'o9_39967', 'o9_40361', 'o9_40519', 'o9_40873', 'o9_41312', 'o9_41909', 
    'o9_42515', 'o9_42735', 'o9_43407', 'o9_43446'
]

def maybe_negate(x, flip):
    return -x if (x is not None and flip) else x

def get_variant_invariants(k2, flip):
    return {
        's-invariant': maybe_negate(s_dict.get(k2), flip),
        'signature': maybe_negate(sign_dict.get(k2), flip),
        'tau': maybe_negate(tau_dict.get(k2), flip),
        'epsilon': maybe_negate(eps_dict.get(k2), flip)
    }

### ---- Generate All Pairs ----

all_pairs = [(k1, k2, variant) for k1, k2 in combinations_with_replacement(names, 2) for variant, _ in variant_types]
print(f"Total pairs: {len(all_pairs)}")

# Filter out slice knots, since obviously, all slice knots will be concordant
non_slice_pairs = [(k1, k2, v) for (k1, k2, v) in all_pairs if k1 not in slice_census and k2 not in slice_census]
print(f"After removing slice knots: {len(non_slice_pairs)}")

# Remove trivial identity pairs
filtered_pairs = [(k1, k2, v) for (k1, k2, v) in non_slice_pairs if not (k1 == k2 and v == 'identity')]
print(f"After removing (k,k,'identity') pairs: {len(filtered_pairs)}")

### ---- Classification ----

not_smoothly_concordant = []
not_topologically_concordant = []
possibly_smoothly_concordant = []
possibly_topologically_concordant = []

invariant_dicts = {
    's-invariant': s_dict,
    'tau': tau_dict,
    'epsilon': eps_dict,
    'signature': sign_dict
}

for k1, k2, variant in filtered_pairs:
    flip = dict(variant_types)[variant]
    k2_inv = get_variant_invariants(k2, flip)

    # Check all invariants for smooth concordance (including signature!)
    smooth_fail = None
    for inv_name in ['s-invariant', 'tau', 'epsilon', 'signature']:
        val1 = invariant_dicts[inv_name].get(k1)
        val2 = k2_inv[inv_name]
        if val1 is not None and val2 is not None and val1 != val2:
            smooth_fail = f"{inv_name}: {val1} vs {val2}"
            break

    if smooth_fail:
        not_smoothly_concordant.append((k1, k2, variant, smooth_fail))
    else:
        possibly_smoothly_concordant.append((k1, k2, variant))

    # For topological concordance, only signature matters
    sig1 = sign_dict.get(k1)
    sig2 = k2_inv['signature']
    if sig1 is not None and sig2 is not None and sig1 != sig2:
        not_topologically_concordant.append((k1, k2, variant, f"signature: {sig1} vs {sig2}"))
    else:
        possibly_topologically_concordant.append((k1, k2, variant))

### ---- Print Final Counts ----

print(f"\nFinal Counts:")
print(f"✔️ Possibly smoothly concordant: {len(possibly_smoothly_concordant)}")
print(f"❌ Not smoothly concordant: {len(not_smoothly_concordant)}")
print(f"✔️ Possibly topologically concordant: {len(possibly_topologically_concordant)}")
print(f"❌ Not topologically concordant: {len(not_topologically_concordant)}")

Total pairs: 3213112
After removing slice knots: 2916112
After removing (k,k,'identity') pairs: 2914905

Final Counts:
✔️ Possibly smoothly concordant: 29537
❌ Not smoothly concordant: 2885368
✔️ Possibly topologically concordant: 64037
❌ Not topologically concordant: 2850868


In [8]:
not_topologically_concordant[524]

('m004', 'v2090', 'identity', 'signature: 0 vs 60')

In [9]:
not_smoothly_concordant[4321]

('m004', 'o9_40449', 'reverse', 'tau: 0 vs 1')

In [ ]:
import time  # <-- Add this at the top with other imports if not already imported

### ---- Further Obstruct with HKL ----

hkl_obstructed = []
still_possibly_topological = []
still_possibly_smooth = []

for index, (k1, k2, variant) in enumerate(possibly_topologically_concordant[20627:], 1):
    print(f"\n🔄 [{index}/{len(possibly_topologically_concordant)}] Working on: {k1}, {k2}, {variant}")
    
    start_time = time.time()  # <-- Start timing here

    try:
        w1 = braid_dict[k1]
        if variant == 'identity':
            w2 = braid_dict[k2]
        elif variant == 'reverse':
            w2 = reverse_orientation(braid_dict[k2])
        elif variant == 'mirror':
            w2 = mirror(braid_dict[k2])
        elif variant == 'reverse_mirror':
            w2 = reverse_orientation(mirror(braid_dict[k2]))
        else:
            continue

        w = connected_sum(w1, reverse_orientation(mirror(w2)))
        K = snappy.Link(braid_closure=w)
        K.simplify('global')
        M = K.exterior()
        M.randomize()

        success = False
        max_attempts = 5
        for attempt in range(max_attempts):
            try:
                print(f"⏳ Attempt {attempt+1} running HKL...")
                obstr = M.slice_obstruction_HKL([(20, [0, 20])], verbose=True)
                if isinstance(obstr, tuple):
                    hkl_obstructed.append((k1, k2, variant, f"HKL obstruction {obstr}"))
                    print("🟩 Obstruction found!", obstr)
                else:
                    still_possibly_topological.append((k1, k2, variant))
                    still_possibly_smooth.append((k1, k2, variant))
                    print("🟥 No obstruction.")
                success = True
                break
            except Exception as e:
                print(f"⚠️ Attempt {attempt+1} failed: {e}")
                M.randomize()

        if not success:
            print(f"❌ HKL permanently failed for {k1}, {k2}, {variant}")
            still_possibly_topological.append((k1, k2, variant))
            still_possibly_smooth.append((k1, k2, variant))

    except Exception as e:
        print(f"💥 Critical setup failure for {k1}, {k2}, {variant}: {e}")
        still_possibly_topological.append((k1, k2, variant))
        still_possibly_smooth.append((k1, k2, variant))

    elapsed = time.time() - start_time  # <-- End timing here
    print(f"⏱️ Time taken: {elapsed:.2f} seconds")

    # Save results after each iteration
    with open("hkl_obstructed_2.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Knot1", "Knot2", "Variant", "Obstruction"])
        writer.writerows(hkl_obstructed)

    with open("still_possibly_topological_2.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Knot1", "Knot2", "Variant"])
        writer.writerows(still_possibly_topological)

    with open("still_possibly_smooth_2.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Knot1", "Knot2", "Variant"])
        writer.writerows(still_possibly_smooth)

### ---- Final Output ----

print("\n🔍 HKL filtering done.")
print(f"🧱 HKL obstructed (not topologically concordant): {len(hkl_obstructed)}")
print(f"🟢 Remaining possibly topologically concordant: {len(still_possibly_topological)}")
print(f"🟢 Remaining possibly smoothly concordant: {len(still_possibly_smooth)}")





🔄 [1/64037] Working on: t00423, o9_04435, identity
⏳ Attempt 1 running HKL...
   Looking at (13, 3) ...
🟩 Obstruction found! (13, 3)
⏱️ Time taken: 477.39 seconds

🔄 [2/64037] Working on: t00423, o9_04435, reverse
⏳ Attempt 1 running HKL...
   Looking at (13, 3) ...
🟩 Obstruction found! (13, 3)
⏱️ Time taken: 925.79 seconds

🔄 [3/64037] Working on: t00423, o9_07943, identity
⏳ Attempt 1 running HKL...
   Looking at (2, 5) ...
🟩 Obstruction found! (2, 5)
⏱️ Time taken: 2.01 seconds

🔄 [4/64037] Working on: t00423, o9_07943, reverse
⏳ Attempt 1 running HKL...
   Looking at (2, 5) ...
🟩 Obstruction found! (2, 5)
⏱️ Time taken: 1.81 seconds

🔄 [5/64037] Working on: t00423, o9_08224, identity
⏳ Attempt 1 running HKL...
   Looking at (3, 5) ...
🟩 Obstruction found! (3, 5)
⏱️ Time taken: 17.03 seconds

🔄 [6/64037] Working on: t00423, o9_08224, reverse
⏳ Attempt 1 running HKL...
   Looking at (3, 5) ...
🟩 Obstruction found! (3, 5)
⏱️ Time taken: 5.68 seconds

🔄 [7/64037] Working on: t00423, o